Reading from recordings

In [70]:
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset
import numpy as np
import processing
from torch.utils.data import DataLoader, random_split


class SensorDataset(Dataset):
    def __init__(self):
        self.samples = []

        recording_list = [
            {'title': 'recording_2022_05_25_11_55_19', 'start': 300, 'end': 400, 'label': 0},  # 0 people
            {'title': 'recording_2022_05_25_11_46_26', 'start': 300, 'end': 400, 'label': 1},  # 1 person
            {'title': 'recording_2022_05_25_11_47_20', 'start': 300, 'end': 400, 'label': 2},  # 2 people
            {'title': 'recording_2022_05_25_11_48_56', 'start': 300, 'end': 400, 'label': 3},  # 3 people
        ]

        for recording in recording_list:
            file_name = f'../../recordings/{recording["title"]}/RadarIfxAvian_00/radar.npy'
            data = np.load(file_name)
            start_frame, end_frame, label = recording["start"], recording["end"], recording["label"]

            # iterate frames
            processed_data = processing.processing_rangeDopplerData(data[start_frame:end_frame])
            processed_data = np.abs(processed_data)
            for f in processed_data:
                self.samples.append((f, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


dataset = SensorDataset()
print(len(dataset))
print(dataset[120])
train_size, test_size = round(len(dataset) * 0.8), round(len(dataset) * 0.2)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

400
(array([[[  0.        , 180.14322919, 414.26175531, ...,  71.40206434,
          80.81485765, 123.06359895],
        [  0.        , 274.92280076, 235.08042928, ...,  48.61058961,
          50.91901018,  34.86425288],
        [  0.        ,  37.32593779, 377.24152753, ..., 105.31151306,
          53.35162562,  34.58918964],
        ...,
        [  0.        ,  61.45960145, 228.19512446, ...,  59.33214726,
          43.12821465, 103.70522056],
        [  0.        , 138.3771356 , 337.33718953, ..., 134.94625284,
          91.94628944,  62.52485891],
        [  0.        , 390.38915245, 295.01424377, ...,  26.47776821,
          94.82267482,  55.38843505]],

       [[  0.        , 784.14347068, 205.3188624 , ...,  61.91715975,
          68.59683895,  84.65631788],
        [  0.        ,  92.91415885, 205.98820719, ...,  67.72016161,
          56.13727272, 103.2191769 ],
        [  0.        , 342.90855188, 611.73461534, ..., 110.20194721,
         116.65829515,  84.3881543 ],
        

In [75]:
input_size = 3 * 64 * 64
hidden_size = [128, 64]
output_size = 4
import torch.nn as nn
from torch import optim


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()

        # Inputs to hidden layer linear transformation
        self.layer1 = nn.Linear(input_size, hidden_size[0])
        # Hidden layer 1 to HL2 linear transformation
        self.layer2 = nn.Linear(hidden_size[0], hidden_size[1])
        # HL2 to output linear transformation
        self.layer3 = nn.Linear(hidden_size[1], output_size)

        # Define relu activation and LogSoftmax output
        self.relu = nn.ReLU()
        self.LogSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # HL1 with relu activation
        out = self.relu(self.layer1(x))
        # HL2 with relu activation
        out = self.relu(self.layer2(out))
        # Output layer with LogSoftmax activation
        out = self.LogSoftmax(self.layer3(out))
        return out


model = NeuralNet(input_size, hidden_size, output_size)
lossFunction = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [74]:
num_epochs = 10
for epoch in range(num_epochs):
    loss_ = 0
    for images, labels in train_loader:
        # Forward Pass
        output = model(images)
        # Loss at each iteration by comparing to target(label)
        loss = lossFunction(output, labels)

        # Backpropagation gradient of loss
        optimizer.zero_grad()
        loss.backward()

        # Updating parameters(weights and bias)
        optimizer.step()

        loss_ += loss.item()
    print("Epoch{}, Training loss:{}".format(epoch, loss_ / len(train_loader)))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (12288x64 and 24576x128)